<a href="https://colab.research.google.com/github/abiflynn/data_analysis_python/blob/main/data_exploration/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning with Pandas


In [1]:
import pandas as pd

In [2]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

## 1.&nbsp; Duplicates

In [3]:
# orders
orders.duplicated().sum()

0

In [4]:
# orderlines
orderlines.duplicated().sum()

0

# 2.&nbsp; `.info()`

In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [6]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [7]:
print(f"5 missing values represents {((orders.total_paid.isna().sum() / orders.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [8]:
orders = orders.loc[~orders.total_paid.isna(), :]

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [9]:
orders["created_date"] = pd.to_datetime(orders["created_date"])

### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [10]:
orderlines["date"] = pd.to_datetime(orderlines["date"])

#### 4.1.2.&nbsp;`unit_price`

In [11]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

ValueError: ignored

When I try to convert `unit_price` to a numerical datatype, I receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points.

In [12]:
orderlines.unit_price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()

False    257814
True      36169
Name: unit_price, dtype: int64

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [13]:
two_dot_percentage = ((orderlines.unit_price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [14]:
two_dot_order_ids_list = orderlines.loc[orderlines.unit_price.astype(str).str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines = orderlines.loc[~orderlines.id_order.isin(two_dot_order_ids_list)]

In [15]:
orderlines.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [16]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

It worked perfectly

# Challenge: Clean the `products` DataFrame


In [17]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [18]:
products.head()

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


## Duplicates

In [19]:
products.duplicated().sum()

8746

In [20]:
products.drop_duplicates(inplace=True)

## `.info()`

In [21]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


### Missing values
We can see from `.info()` above that we have missing values in `desc` and `price`

#### `desc`

In [22]:
products.desc.isna().sum()

7

In [23]:
products.loc[products.desc.isna(), :]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,107,814.659,0,1298
16128,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,1.568.206,1.568.206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,5.659.896,0,12175397
18152,KAN0034-A,Open - Kanex USB-C Gigabit Ethernet Adapter Ma...,NaN,29.99,237.925,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.99,1.441.174,0,11905404


In [24]:
products.loc[products.desc.isna(), "desc"] = products.loc[products.desc.isna(), "name"]

#### `price`

In [25]:
products.price.isna().sum()

46

In [26]:
print(f"The missing values in price are {((products.price.isna().sum() / products.shape[0]) * 100).round(2)}% of all rows in the DataFrame")

The missing values in price are 0.43% of all rows in the DataFrame


In [27]:
products = products.loc[~products.price.isna(), :]

### Data types


#### `price`

First, let's see how many values are affected by the 2 problems mentioned above.

In [28]:
price_problems_number = products.loc[(products.price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products.price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
price_problems_number

542

In [29]:
print(f"The column price has in total {price_problems_number} wrong values. This is {round(((price_problems_number / products.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column price has in total 542 wrong values. This is 5.15% of the rows of the DataFrame


5.15% is a reasonable amount of our data. However, the price column will be important to understanding discounts, so I'd like it to be very trustworthy as we are basing business decisions on it. Therefore, I will delete these rows

In [30]:
products = products.loc[(~products.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~products.price.astype(str).str.contains("\d+\.\d{3,}")), :]

To complete our task, let's convert the column to a numeric datatype

In [31]:
products["price"] = pd.to_numeric(products["price"])

#### `promo_price`

Again, let's begin by seeing how many values are affected by the 2 problems

In [32]:
promo_problems_number = products.loc[(products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
promo_problems_number

9232

In [33]:
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column promo_price has in total 9232 wrong values. This is 92.39% of the rows of the DataFrame


In [35]:
promo_price_df = products.loc[(products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :]
promo_price_df.sample(10)

,sku,name,desc,price,promo_price,in_stock,type
13616,APP1807,Apple Correa Nailon 38mm Braid Navy / Blue Lagoon,Braided nylon strap with four layers of tissue...,59.00,550.006,0,2449
17526,CRU0053-2,Crucial memory Mac 32GB (2x16GB) SO-DIMM DDR4 ...,Crucial memory Mac SO-DIMM 32GB 2400MHz DDR4 i...,423.98,4.229.906,1,1364
2923,SAN0096,SanDisk Ultra 64GB USB Flash Drive Fit 3.0,Pendrive ultracompact 64GB USB 3.0 Mac and PC.,29.90,179.903,0,57445397
15260,LEX0042,Lexar C20c JumpDrive USB Flash Drive 128GB-C /...,USB flash drive with USB 3.1-C and 128GB conne...,71.37,672.808,0,57445397
16227,APP2218,"Apple iMac 27 ""Core i5 Retina 5K 34GHz | 8GB |...",IMac desktop computer 27 inch 5K Retina 8GB RA...,2939.00,27.630.048,0,"5,74E+15"
14879,STA0059,Startech USB-C Cable to VGA 1m Black,1m adapter cable with USB-C reversible VGA con...,62.99,449.902,0,12585395
14674,QNA0204,QNAP TS-231P NAS,2-bay NAS server with 2 LAN ports and a powerf...,216.59,2.069.899,0,12175397
13307,OTT0149,OtterBox iPhone Case Symmetry 8/7 Clear Clear ...,resistant and anti thin sleeve edges beveled b...,29.99,199.904,0,11865403
763,APP0990,Apple Mac Pro 12-core 27GHz | 16GB RAM | 1TB F...,New Mac Pro with 16GB of RAM 12-core GPU 1TB 2...,8969.00,84.375.841,0,1282
636,BEL0139,Belkin Lego iPhone Case SE / 5s / 5 Blue / Purple,Lego rigid shell for iPhone SE / 5s / 5.,29.99,99.946,0,11865403


Over 90% of the data in this column is corrupt. Delete the column.

In [36]:
products_cl = products.drop(columns=["promo_price"])